<center><img src="https://ivybound.net/wp-content/uploads/2016/11/score-300x300.jpg",height='500',width='500'></center>

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Dataset Overview</b></h1>

* School : Name of the school the student is enrolled in.
* School setting : The location of the school.
* School type : The type of school. Either public or non-public.
* Classroom : The type of classroom.
* Teaching method : Teaching methods: Either experimental or Standard.
* n_student : Number of students in the class.
* Student_id : A unique ID for each student.
* Gender : The gender of the students: male or female.
* Lunch : Whether a student qualifies for free/subsidized lunch or not.
* Pretest : The pretest score of the students out of 100.




<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Importing libraries and loading Dataset</b></h1>

In [ ]:
import pandas as pd
import numpy as np
import random
import seaborn as sns
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
data=pd.read_csv("../input/predict-test-scores-of-students/test_scores.csv")
data.head(5)

In [ ]:
#drop the student_id Column
data.drop(['student_id'],axis=1,inplace=True)

In [ ]:
data.isnull().sum()

There are no missing values in any attributes

In [ ]:
#numerical attributes
numeric_columns=data.select_dtypes(include=['number']).columns.tolist()

#categorical attributes
cat_columns=data.select_dtypes(exclude=['number','bool_']).columns.tolist()

In [ ]:
def label_function(val):
    return f'{val / 100 * len(data):.0f}\n{val:.0f}%'

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Visuzlisation</b></h1>

In [ ]:
colors=['tomato', 'gold', 'skyblue','violet', 'lime']
for col in cat_columns:
    col_no=len(pd.unique(data[col]))
    if col_no<4:
        color=(random.sample(colors, col_no))
        data.groupby(col).size().plot(kind='pie', autopct=label_function, textprops={'fontsize': 15},colors=color)
        #plt.xlabel('Per country', size=22)
        #plt.ylabel('Per gender', size=22)
        plt.ylabel("")
        plt.tight_layout()
        plt.title("Distibution of {}".format(col),size=22)
        plt.show()

In [ ]:
pd.crosstab(data['school_setting'],data['school_type'], normalize='index')

In [ ]:
pd.crosstab(data['school_type'],data['lunch'], normalize='index')

The above table shows,Most of the students from non-public school are not qualified for reduced/free lunch.

In [ ]:
pd.pivot_table(data, values='posttest', index=['school_type'],columns=['lunch'], aggfunc=np.mean)


From the above table it is clear that the highest score is from the private school that doesnot qualify for the reduced/free lunch

In [ ]:
f, axes = plt.subplots(1, 3,figsize=(15,8))
sns.boxplot(data=data, x='lunch', y='posttest', ax=axes[0])
#plt.xticks(fontsize=14)
sns.boxplot(data=data, x='school_setting', y='posttest', ax=axes[1])
#plt.xticks(fontsize=14)
sns.boxplot(data=data, x='school_type', y='posttest', ax=axes[2])
plt.tight_layout()
#plt.xticks(fontsize=14)
plt.show()

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,8))
sns.boxplot(data=data, x='gender', y='posttest', ax=axes[0])
sns.boxplot(data=data, x='teaching_method', y='posttest', ax=axes[1])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(data=data, x='n_student', y='posttest',hue='school_type')
plt.show()

In [ ]:
new_data=[]
for col in numeric_columns:
    if col=='n_student':
        continue
    new_data.append(data[col].tolist())

fig = plt.figure(figsize =(15,5))
ax = fig.add_subplot(111)

# Creating axes instance
bp = ax.boxplot(new_data, patch_artist = True,
                notch ='True', vert = 0)

colors = ['#0000FF', '#00FF00']

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

# changing color and linewidth of
# whiskers
for whisker in bp['whiskers']:
    whisker.set(color ='#8B008B',
                linewidth = 1.5,
                linestyle =":")

# changing color and linewidth of
# caps
for cap in bp['caps']:
    cap.set(color ='#8B008B',
            linewidth = 2)

# changing color and linewidth of
# medians
for median in bp['medians']:
    median.set(color ='red',
               linewidth = 3)

# changing style of fliers
for flier in bp['fliers']:
    flier.set(marker ='D',
              color ='#e7298a',
              alpha = 0.5)

# x-axis labels
ax.set_yticklabels(['pretest','posttest'])

# Adding title
plt.title("Customized box plot")

# Removing top axes and right axes
# ticks
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()

# show plot
plt.show(bp)

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Transforming Categorical to Numerical values</b></h1>

In [ ]:
label_encoder = LabelEncoder()
for col in cat_columns:
    data[col]=label_encoder.fit_transform(data[col])


In [ ]:
data

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Normalizing Numerical values</b></h1>

In [ ]:
for col in numeric_columns:
    data[col]= MinMaxScaler().fit_transform(np.array(data[col]).reshape(-1,1))

In [ ]:
target=data['posttest']
data=data.drop(['posttest'],axis=1)
data.head(5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data,target, test_size = 0.25)

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Linear Regression</b></h1>

In [ ]:
regression = LinearRegression()
regression.fit(X_train, y_train)
print(regression.score(X_test, y_test))
y_pred = regression.predict(X_test)

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Evaluvation</b></h1>

In [ ]:
def evaluvate(y_test,y_pred):
    result={}
    result['MAE'] = round(mean_absolute_error(y_test, y_pred),4)
    result['MSE'] = round(mean_squared_error(y_test, y_pred),4)
    result['r2'] = round(r2_score(y_test, y_pred),4)
    return result
    

In [ ]:
result=evaluvate(y_test,y_pred)


In [ ]:
print("{:<25} {:<15}".format('Model','Accuracy'))
for k, v in sorted(result.items(), key=lambda item :item[1],reverse=True):
    
    print("{:<25} {:<15}".format(k,v))

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Polynomial Regression</b></h1>

In [ ]:
 poly = PolynomialFeatures(degree = 2)
X_poly = poly.fit_transform(data)
X_train, X_test, y_train, y_test = train_test_split(X_poly,target, test_size = 0.25)
poly.fit(X_train, y_train)
poly_reg = LinearRegression()
poly_reg.fit(X_train, y_train)
print(poly_reg.score(X_test, y_test))
y_pred = poly_reg.predict(X_test)

<h1 style="background-color:MediumSeaGreen;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>Evaluvation</b></h1>

In [ ]:
def evaluvate(y_test,y_pred):
    result={}
    result['MAE'] = round(mean_absolute_error(y_test, y_pred),4)
    result['MSE'] = round(mean_squared_error(y_test, y_pred),4)
    result['r2'] = round(r2_score(y_test, y_pred),4)
    return result
    

In [ ]:
result=evaluvate(y_test,y_pred)


In [ ]:
print("{:<25} {:<15}".format('Model','Accuracy'))
for k, v in sorted(result.items(), key=lambda item :item[1],reverse=True):
    
    print("{:<25} {:<15}".format(k,v))

# If you find this notebook useful
<h1 style="background-color:tomato;font-family:newtimeroman;font-size:500%;text-align:center;border-radius: 15px 50px;padding: 5px 5px 5px 5px">
    <b>feel free to upvote!</b></h1>